# [Spinal Cord Toolbox (SCT)](https://spinalcordtoolbox.com/index.html)

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/overview.png" align="center" width="750px">

## Installation Instructions
The instructions below are for MacOS and Linux platforms. \
For more installation instructions, such as installing on Windows or Docker, check [here](https://spinalcordtoolbox.com/user_section/installation.html).

### Run the following (starting with $) in your terminal
Skip if you already have SCT and FSLeyes installed.

In [ ]:
# First we'll clone the SCT repository from GitHub 
$git -C spinalcordtoolbox pull || git clone https://github.com/spinalcordtoolbox/spinalcordtoolbox.git spinalcordtoolbox

# Change our directory to the newly created directory
$cd spinalcordtoolbox

#start installation
$./install_sct -y

In [ ]:
# Install the following
!${SCT_DIR}/python/envs/venv_sct/bin/pip install matplotlib-inline
!${SCT_DIR}/python/envs/venv_sct/bin/pip install Ipython

Open a new terminal, and run the following to install FSLeyes.\
[FSLeyes](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLeyes) is a GUI image viewer. We'll need this to view our input/output images, and perform labelling when necessary


In [ ]:
# Run the following to install fsleyes
$source spinalcordtoolbox/python/etc/profile.d/conda.sh
$conda create -c conda-forge -p ~/fsleyes_env fsleyes -y

# Next let's make fsleyes available in the terminal without having to activate its environment
$sudo ln -s ~/fsleyes_env/bin/fsleyes /usr/local/bin/fsleyes

Test if SCT and FSLeyes are successfully installed, and working.

In [ ]:
# Run the following in the terminal
$sct_testing

# Run the following to check for fsleyes...
$fsleyes

In [ ]:
# library to open our results .html instead of from the terminal
import webbrowser

# [Spinal Cord Segmentation - Tutorial](https://spinalcordtoolbox.com/user_section/tutorials/segmentation.html)

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/spinalcord-segmentation/spinalcord-segmentation.png" align="center" width="400px">

Without data there's no SCT, so let's download one for this tutorial by running the following.

In [ ]:
# If you're on linux run
!wget https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_spinalcord-segmentation.zip -O data_spinalcord-segmentation.zip -q

# Otherwise, if you're on MacOS run
# !curl -L https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_spinalcord-segmentation.zip -o data_spinalcord-segmentation.zip

!unzip -o data_spinalcord-segmentation.zip -d spinalcord_seg && rm data_spinalcord-segmentation.zip

In [ ]:
%cd 'spinalcord_seg/single_subject/data'
pwd = %pwd

## Algorithm #1: **sct_propseg**

This is a machine learning based method that does the following:
* Step 1️⃣: **Centerline detection** - the first step is detecting the centerline of the spinal (Click this [OptiC](https://archivesic.ccsd.cnrs.fr/PRIMES/hal-01713965v1)). This can also be achieve with **sct_get_centerline**
* Step 2️⃣: **Mesh propagation** - here, the algorithm propagates along the spinal cord, and creates a 3D mesh
* Step 3️⃣: **Surface refinement** - finally the algorithm refines the mesh created in step 2

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/spinalcord-segmentation/optic_steps.png" align="center" width="550px" height="450px"> <img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/spinalcord-segmentation/mesh_propagation.png" align="center" width="550px" height="450px">

Input arguments:
* -i : Input image
* -c : Contrast of the input image
* -qc : Directory for Quality Control reporting. QC reports allow us to evaluate the segmentation slice-by-slice \

Output files/folders:
* t2_seg.nii.gz : 3D binary mask of the segmented spinal cord

### Hands-on: Using sct_propseg on T2 data

In [ ]:
!sct_propseg -i t2/t2.nii.gz -ofolder ./propseg -c t2 -qc ./propseg/qc_singleSub/t2

In [ ]:
url = "file://" + pwd + "/propseg/qc_singleSub/t2/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2/t2.nii.gz -cm greyscale ./propseg/t2_seg.nii.gz -cm red -a 100.0

### Hands-on: Using sct_propseg on T1 data

In [ ]:
!sct_propseg -i t1/t1.nii.gz -ofolder ./propseg -c t1 -qc ./propseg/qc_singleSub/t1

In [ ]:
url = "file://" + pwd + "/propseg/qc_singleSub/t1/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t1/t1.nii.gz -cm greyscale ./propseg/t1_seg.nii.gz -cm red -a 100.0

## Algorithm #2: **sct_deepseg_sc**

Unlike **sct_propseg**, this algorithm is based on deep learning. **sct_deepseg_sc** does the following:
* Step 1️⃣: **Spinal cord detection** - a convolutional neural network (CNN) is used to generate a probablistic heatmap of the location of the spinal cord
* Step 2️⃣: **Centerline detection** - The heatmap is then fed into the OptiC algorithm (mentioned previoulsy) to detect the spinal cord centerline.
* Step 3️⃣: **Patch extraction** - The spinal cord centerline is used to extract a patch (useful region) from the image.
* Step 4️⃣: **Segmentation** - finally, a second CNN is applied to the extracted patch to segment the spinal cord.


<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/spinalcord-segmentation/sct_deepseg_sc_steps.png" align="center" width="650px" height="450px">

Input arguments:
* -i : Input image
* -c : Contrast of the input image
* -ofolder : The folder to output files to. We specify this here so that we don’t overwrite the t2_seg.nii.gz file that was produced by sct_propseg.
* -qc : Directory for Quality Control reporting. QC reports allow us to evaluate the segmentation slice-by-slice \

Output files/folders:
* t2_seg.nii.gz : 3D binary mask of the segmented spinal cord

### Hands-on: Using sct_deepseg_sc on T1 data

In [ ]:
!sct_deepseg_sc -i t1/t1.nii.gz -ofolder ./deepseg -c t1 -qc ./deepseg/qc_singleSub/t1

In [ ]:
url = "file://" + pwd + "/deepseg/qc_singleSub/t1/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t1/t1.nii.gz -cm greyscale ./deepseg/t1_seg.nii.gz -cm red -a 70.0

## Congratulations for completing this tutorial 🍾💐